In [ ]:
import numpy as np
import pandas as pd 
from typing import List 
from sklearn.preprocessing import (
  LabelEncoder, 
  StandardScaler
)
from sklearn.model_selection import (
  train_test_split,
  GridSearchCV, 
  StratifiedKFold
)
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
  classification_report,
  accuracy_score,
  precision_score,
  recall_score,
  f1_score,
  confusion_matrix,
  roc_auc_score,
  roc_curve
)
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Modelos 
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC 

PATH = "../_data/healthcare_dataset_stroke_data/healthcare-dataset-stroke-data.csv"

# semilla para reproducibilidad; None para aleatoriedad completa
random_state = 42

## Stroke Prediction Dataset

**Descargar el siguiente dataset**: [Link](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset)

**Características del Dataset**:

- `id`: identificador único
- `gender`: "Male", "Female" u "Other"
- `age`: edad del paciente
- `hypertension`: 0 si el paciente no tiene hipertensión, 1 si el paciente tiene hipertensión
- `heart_disease`: 0 si el paciente no tiene enfermedades cardíacas, 1 si el paciente tiene una enfermedad cardíaca
- `ever_married`: "Yes" o "No"
- `work_type`: "children", "Govt_jov", "Never_worked", "Private" o "Self-employed"
- `residence_type`: "Rural" o "Urban"
- `avg_glucose_level`: nivel promedio de glucosa en sangre
- `bmi`: índice de masa corporal
- `smoking_status`: "formerly smoked", "never smoked", "smokes" o "Unknown"
- `stroke`: 1 si el paciente tuvo un ACV o 0 si no

**Objetivo**: Predecir si es probable que un paciente tenga un ataque al corazón (stroke)

### Preprocesamiento del Dataset

In [ ]:
category_features = [ 'gender', 'smoking_status', 'residence_type', 'work_type' ]

df = pd.read_csv(PATH)
df.columns = df.columns.str.lower() 

df["ever_married"] = df["ever_married"].map({"Yes": True, "No": False})

for name_feature in category_features:
  df[name_feature] = df[name_feature].astype('category')

df = df.dropna().reset_index(drop=True)
display(df.head(10))

Ahora, se va a codificar las variables categóricas que están representadas como cadenas de texto usando el encoder `LabelEncoder`, que codifica las cadenas de texto como números. 

In [ ]:
for name_feature in category_features:
  encoder = LabelEncoder()
  df[name_feature] = encoder.fit_transform(df[name_feature])

display(df.head(10))

In [ ]:
category_features = [ 
  'gender', 'hypertension', 'heart_disease', 'ever_married',
  'smoking_status', 'residence_type', 'work_type', 'stroke'
]
for column in category_features:
  display(df[column].value_counts())

Se tiene un dataset altamente desbalanceado

In [ ]:
# Función para separar características de la variable objetivo
def get_Xy(df:pd.DataFrame, target:str):
  X = df.drop(columns=[target])
  y = df[target]
  return X, y

### Análisis Exploratorio de Datos

El **Análisis Exploratorio de Datos** (EDA) es una fase donde se examinan y se comprende la estructura, características y relaciones subyacentes en un dataset. Este proceso permite:
- *Comprensión estructural*: Evaluar dimensiones, tipos de datos y distribuciones para guiar la preparación de datos.
- *Identificación de relaciones*: Descubrir patrones, tendencias y correlaciones entre variables
- *Detección de anomalías*: Reconocer valores atípicos, datos faltantes o inconsistencias
- *Información para modelado*: Fundamentar decisiones sobre transformaciones, ingeniería de características y selección de algoritmos.

Esta exploración sistemática establece las bases para un pipeline de ML robusto y con resultados interpretables.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Distribución de la Variable Target 
sns.countplot(data=df, x='stroke', ax=axes[0,0])
axes[0,0].set_title('Distribución de Strokes')

# Age vs Stroke
sns.boxplot(data=df, x='stroke', y='age', ax=axes[0,1])
axes[0,1].set_title('Edad vs Stroke')

# Avg Glucose Level vs Stroke
sns.violinplot(data=df, x='stroke', y='avg_glucose_level', ax=axes[0,2])
axes[0,2].set_title('Glucosa vs Stroke')

# BMI vs. Stroke
sns.boxplot(data=df, x='stroke', y='bmi', ax=axes[1,0])
axes[1,0].set_title('BMI vs Stroke')

# Gender vs. Stroke
sns.countplot(data=df, x='gender', hue='stroke', ax=axes[1,1])
axes[1,1].set_title('Género vs Stroke')

# Work-Type vs. Stroke
sns.countplot(data=df, x='work_type', hue='stroke', ax=axes[1,2])
axes[1,2].tick_params(axis='x')
axes[1,2].set_title('Trabajo vs Stroke')

plt.tight_layout()
plt.show()

In [ ]:
numeric_df = df.select_dtypes(include=[np.number]).drop(columns=['id'])

plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(numeric_df.corr(), dtype=bool))

sns.heatmap(numeric_df.corr(), mask=mask, annot=True, cmap='RdBu', fmt=".2f")
plt.title('Matriz de Correlación')
plt.tight_layout()
plt.show()

### Clasificación usando Diferentes Modelos

En un conjunto de datos desbalanceado, la Exactitud (Accuracy) es engañosa, ya que el modelo puede obtener una puntuación alta simplemente prediciendo siempre la clase mayoritaria. Por lo que se sugiere para este dataset usar AUC (Area Under the Curve), la cual es una medida de la habilidad del clasificador para distinguir entre las clases y se usa como una representación de la curva ROC. Cuando mayor sea el AUC, mejor será el rendimiento del modelo para distinguir entre las clases positivas y negativas. Una curva ROC (Receiver Operating Characteristic Curve) es un gráfico que muestra el rendimiento de un modelo de clasificación en todos los umbrales de clasificación.

**Modelos Propuestos**: SVM (Support Vector Machine), Gaussian Naive Bayes, K-Neighbors, Decision Tree, Random Forest, Logistic Regression


In [ ]:
X,y = get_Xy(df, target='stroke')
display(X.head(10))
display(f"Dimensión de Y: {y.shape[0]}")

In [ ]:
# división del dataset en entrenamiento y prueba 
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=random_state, stratify=y
)

# información de las particiones
print(f"Dimensión de X_train: {X_train.shape}")
print(f"Dimensión de X_test: {X_test.shape}")
print(f"Dimensión de y_train: {y_train.shape}")
print(f"Dimensión de y_test: {y_test.shape}")

### Baseline

Un **Baseline** es un modelo de referencia simple que sirve como punto de comparación mínimo para evaluar modelos más complejos. Este modelo permite:
- Establecer un mínimo de rendimiento que cualquier modelo puede superar.
- Detectar problemas en los datos o en la métrica de evaluación.
- Medir el valor agregado de modelos más complejos. 

Algunos de los tipos comunes de baselines son: *clase mayoritaria* (siempre predice la clase más frecuente), *clase minoritaria* (siempre predice la clase menos frecuente), *clase aleatoria* (predice aleatoriamente según la distribución de clases) y *clase constante* (siempre predice un valor fijo) 

In [ ]:
def create_negative_baseline(y_train, X_test, y_test):
  "Baseline que siempre predice la clase negativa"
  # Siempre predecir 0 para todas las muestras
  y_pred_baseline = np.zeros(len(y_test), dtype=int)
  # Para AUC se necesitan probabilidades (0% de probabilidad de stroke)
  y_pred_proba_baseline = np.zeros(len(y_test))
  
  # Calcular métricas
  accuracy = accuracy_score(y_test, y_pred_baseline)
  auc_roc = roc_auc_score(y_test, y_pred_proba_baseline)
  
  return accurancy, auc_roc, y_pred_baseline

accuracy_baseline, auc_roc_baseline, y_pred_baseline = create_negative_baseline(y_train, X_test, y_test)
print(f"Baseline - Accuracy: {accuracy_baseline:.4f} | AUC-ROC: {auc_roc_baseline:.4f}")  

In [ ]:
# Creación de Baseline usando DummyClassifier 
baseline_negative = DummyClassifier(strategy='constant', constant=0)
baseline_negative.fit(X_train, y_train) # Se entrena, pero solo "aprende" a predecir 0

# Predicciones 
y_pred_baseline = baseline_negative.predict(X_test)
# Para AUC se necesitan probabilidades
y_pred_proba_baseline = baseline_negative.predict_proba(X_test)[:, 1]
# Calcular métricas
accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
auc_roc_baseline = roc_auc_score(y_test, y_pred_proba_baseline)

print(f"DummyClassifier Baseline - Accuracy: {accuracy_baseline:.4f} | AUC-ROC: {auc_roc_baseline:.4f}")

#### Modelo: SVM

In [ ]:
# 1. Definición del Pipeline para SVM
svm_pipeline = Pipeline([
  ('scaler', StandardScaler()), 
  ('SVM', SVC(random_state=random_state, probability=True))
])
# 2. Parámetros para Búsqueda de Hiperparámetros
param_grid = {
  'SVM__C': [0.1, 1, 10],              # Parámetro de regularización
  'SVM__kernel': ['rbf','linear'],          # Tipos de kernel
  'SVM__gamma': ['scale', 'auto'],          # Parámetro gamma para kernel RBF
  'SVM__class_weight': ['balanced']         # Manejo de clases desbalanceadas
}
# 3. Entrenamiento con Validación Cruzada 
## Usando StratifiedKFold para mantener proporción de clases en cada fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

# Búsqueda de Hiperparámetros con GridSearchCV
grid_search = GridSearchCV(
  estimator = svm_pipeline,
  param_grid = param_grid,
  cv = cv, 
  scoring = 'roc_auc',          # uso de AUC-ROC por el desbalance de clases
  verbose = 1
)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
# Resultados de la Búsqueda de Hiperparámetros
best_params = grid_search.best_params_
for key, value in best_params.items():
  print(f"{key}: {value}")
print(f"Mejor score de validación (AUC-ROC): {grid_search.best_score_}")

In [ ]:
# Obtener el mejor modelo
best_model = grid_search.best_estimator_
# Hacer predicciones
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
# Calcular métricas de evaluación
accurancy = accuracy_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred_proba)

print(f"Accuracy: {accurancy:.4f} | AUC-ROC: {auc_roc:.4f}")

In [ ]:
# Función útil para evaluar cualquier modelo 
def evaluate_model(model, X_test, y_test):
  "Evalúa un modelo y retorna las métricas principales"
  # Predicciones 
  y_pred = model.predict(X_test)
  
  # Obtener probabilidades si el modelo las soporta 
  if hasattr(model, 'predict_proba'):
    y_predic_proba = model.predict_proba(X_test)[:, 1]
    auc_roc = roc_auc_score(y_test, y_predic_proba)
  else:
    auc_roc = None
  
  # Calcular métricas
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, zero_division=0)
  recall = recall_score(y_test, y_pred, zero_division=0)
  f1 = f1_score(y_test, y_pred, zero_division=0)
  
  return {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'auc_roc': auc_roc
  }
#display(evaluate_model(best_model, X_test, y_test))

In [ ]:
# Curva ROC
FPR, TPR, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(FPR, TPR, color='blue', label='Curva ROC')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Clasificador Aleatorio')
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC del Modelo SVM')
plt.legend()
plt.show()

In [ ]:
# Matriz de Confusión 
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
  xticklabels=['No Stroke', 'Stroke'], 
  yticklabels=['No Stroke', 'Stroke'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Matriz de Confusión')
plt.show()